**Library Imports**

In [1]:
import pandas as pd
import io
import requests
import numpy as np
import os
import seaborn as sns

from scipy.stats import zscore

from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential, load_model

from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

import tensorflow as tf

Panda Preferences

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

Read CSV

In [3]:
df_train = pd.read_csv("../../Datasets/Farm-Flow_Train.csv")
display(df_train)

,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,fwd_header_size_max,bwd_header_size_tot,bwd_header_size_min,bwd_header_size_max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,bwd_pkts_payload.tot,bwd_pkts_payload.avg,flow_pkts_payload.tot,flow_pkts_payload.avg,fwd_iat.tot,fwd_iat.avg,bwd_iat.tot,bwd_iat.avg,flow_iat.tot,flow_iat.avg,traffic,pkts_difference,data_pkts_difference,is_attack
0,0.0,-0.067841,0.005959,0.881829,1.503112,0.000026,0.000041,0.000038,0.194444,192,1.566169,1.478170,256,2.712843,4.268354,-0.000945,-0.000945,1.589631,6.736335,-0.000945,-0.000945,0.829308,0.245222,1.220854,2.096534,0.824683,0.006424,HTTP_Flood,-0.094869,0.139609,1
1,0.0,-0.067841,-0.237735,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,24,0.655956,0.605343,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,Port_Scanning,0.007982,-0.013989,1
2,0.0,-0.067841,-0.393700,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,8,-1.164469,-1.140311,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,UDP_Flood,0.007982,-0.013989,1
3,0.0,-0.067841,-0.393700,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,8,-1.164469,-1.140311,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,UDP_Flood,0.007982,-0.013989,1
4,0.0,-0.067841,-0.393700,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,8,-1.164469,-1.140311,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,UDP_Flood,0.007982,-0.013989,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839594,0.0,-0.067841,-0.276726,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,20,0.200850,0.168930,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,TCP_Flood,0.007982,-0.013989,1
839595,0.0,-0.067841,-0.276726,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,20,0.200850,0.168930,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,TCP_Flood,0.007982,-0.013989,1
839596,0.0,-0.067841,-0.276726,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,20,0.200850,0.168930,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,ICMP_Flood,0.007982,-0.013989,1
839597,0.0,-0.067841,-0.393700,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,8,-1.164469,-1.140311,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,UDP_Flood,0.007982,-0.013989,1


In [4]:
df_test = pd.read_csv("../../Datasets/Farm-Flow_Test.csv")
display(df_test)

,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,fwd_header_size_max,bwd_header_size_tot,bwd_header_size_min,bwd_header_size_max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,bwd_pkts_payload.tot,bwd_pkts_payload.avg,flow_pkts_payload.tot,flow_pkts_payload.avg,fwd_iat.tot,fwd_iat.avg,bwd_iat.tot,bwd_iat.avg,flow_iat.tot,flow_iat.avg,traffic,pkts_difference,data_pkts_difference,is_attack
0,0.0,-0.067841,1.692323,0.881829,1.503112,0.000085,0.000116,0.000116,0.166667,456,1.566169,2.350997,456,2.712843,3.183130,-0.000945,-0.000945,1.589631,3.342022,-0.000945,-0.000945,0.545786,-0.054924,0.848904,0.625086,0.541274,-0.112275,HTTP_Flood,-0.043444,0.446804,1
1,0.0,-0.067841,-0.276726,0.881829,1.189272,0.730159,0.992063,0.992063,0.166667,20,0.200850,0.168930,24,1.923476,1.736165,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163659,-0.167355,TCP_Flood,-0.043444,-0.013989,1
2,0.0,-0.067841,-0.276726,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,20,0.200850,0.168930,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,TCP_Flood,0.007982,-0.013989,1
3,0.0,-0.067841,-0.276726,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,20,0.200850,0.168930,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,TCP_Flood,0.007982,-0.013989,1
4,0.0,-0.067841,-0.393700,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,8,-1.164469,-1.140311,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,UDP_Flood,0.007982,-0.013989,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279862,0.0,-0.067841,-0.237735,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,24,0.655956,0.605343,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,Port_Scanning,0.007982,-0.013989,1
279863,0.0,-0.067841,-0.393700,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,8,-1.164469,-1.140311,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,UDP_Flood,0.007982,-0.013989,1
279864,0.0,-0.067841,-0.276726,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,20,0.200850,0.168930,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,TCP_Flood,0.007982,-0.013989,1
279865,0.0,-0.067841,-0.276726,-0.500297,-0.536844,0.000000,0.000000,0.000000,0.000000,20,0.200850,0.168930,0,-0.444625,-0.434283,-0.000945,-0.000945,-0.019102,-0.052290,-0.000945,-0.000945,-0.159412,-0.168388,-0.076153,-0.096974,-0.163665,-0.167367,MQTT_Flood,0.007982,-0.013989,1


In [5]:
df_train.groupby('is_attack')['is_attack'].count()

is_attack
0      5321
1    834278
Name: is_attack, dtype: int64

In [6]:
df_test.groupby('is_attack')['is_attack'].count()

is_attack
0      1774
1    278093
Name: is_attack, dtype: int64

---------------------------------------

**Create Model & Train Model**

In [7]:
df_train = df_train.drop('traffic', axis=1)
df_test = df_test.drop('traffic', axis=1)

In [8]:
x_columns = df_train.columns.drop('is_attack')

le = LabelEncoder()
le.fit(df_train["is_attack"].values)

x = df_train[x_columns].values
y = df_train["is_attack"].values
y = le.transform(y)

In [9]:
x_columns_val = df_test.columns.drop('is_attack')

le = LabelEncoder()
le.fit(df_test["is_attack"].values)

x_val = df_test[x_columns].values
y_val = df_test["is_attack"].values
y_val = le.transform(y_val)

In [10]:
# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

In [11]:
x_test.shape,y_test.shape

((209900, 29), (209900,))

In [12]:
x_train.shape,y_train.shape

((629699, 29), (629699,))

In [13]:
x_val.shape, y_val.shape

((279867, 29), (279867,))

In [14]:
import numpy as np

# Random state definition
random_state=42
np.random.seed(random_state)

In [15]:
from sklearn import metrics

# Classification Metrics: Accuracy, Recall, Precision & F1Score
def compute_metrics(y_test, pred):
    print('Accuracy:', metrics.accuracy_score(y_test, pred))
    print('Recall:', metrics.recall_score(y_test, pred, average = 'weighted'))
    print('Precision:', metrics.precision_score(y_test, pred, average = 'weighted'))
    print('F1-Score:', metrics.f1_score(y_test, pred, average = 'weighted'))

In [16]:
# Extra Trees Classifier
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

def extra_tree_classifier(x_train, y_train, x_columns):
    # Usage of ExtraTreesClassifier for feature selection
    extra_tree_forest = ExtraTreesClassifier(n_estimators=5, criterion='entropy', max_features=2, random_state=random_state)
    extra_tree_forest.fit(x_train, y_train)
    feature_importances = extra_tree_forest.feature_importances_
    feature_importance_normalized = np.std([tree.feature_importances_ for tree in  extra_tree_forest.estimators_], axis = 0)

    # Plot the feature importances
    plt.rcParams["figure.figsize"] = (70, 40)
    plt.bar(x_columns, feature_importance_normalized, align='center')
    plt.xlabel('Feature Index')
    plt.ylabel('Feature Importance')
    plt.title('Comparison of different feature importances in the current dataset')
    plt.xticks(rotation=90)
    plt.show()

In [17]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

def conf_matrix(y_test, prediction, array_dimension = 2):
    if (array_dimension == 2):
        predict_classes = np.argmax(prediction, axis = 1)
        expected_classes = np.argmax(y_test, axis = 1)
    elif (array_dimension == 1):
        predict_classes = prediction
        expected_classes = y_test
    
    cm = confusion_matrix(expected_classes, predict_classes)
    cmd = ConfusionMatrixDisplay(cm, display_labels=['Normal', 'Malicious'])

    # Plot size
    fig, ax = plt.subplots(figsize=(4,4))
    
    # Add axis labels and rotate them
    ax.set_xlabel('Predicted labels', rotation=0, labelpad=10, fontsize=11)
    ax.set_ylabel('True labels', rotation=90, labelpad=10, fontsize=11)
    
    cmd.plot(ax=ax, cmap=plt.cm.Blues, colorbar=False)

    ax.set_xticks([0, 1])
    ax.set_yticks([0, 1])
    ax.set_xticklabels(['Normal', 'Malicious'])
    ax.set_yticklabels(['Normal', 'Malicious'])
    
    ax.tick_params(axis='x', pad=10)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0, ha='center', va='center')
    ax.tick_params(axis='y', pad=10)
    ax.set_yticklabels(ax.get_yticklabels(), rotation=90, ha='center', va='center')
    
    # Adjust colorbar size
    cax = plt.gcf().axes[-1]
    cax.tick_params(labelsize=11)  # Adjust the font size of colorbar labels
    cax = fig.add_axes([ax.get_position().x1+0.03,ax.get_position().y0,0.02,ax.get_position().height])
    plt.colorbar(cmd.im_, cax=cax)    

In [18]:
def plot_confusion_matrix(cm, target_names, title='Confusion matrix', cmap=None, normalize=False):
    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshowac(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=90)
        plt.yticks(tick_marks, target_names)
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
%%time

# Algorithm: LOGISTIC REGRESSION
import numpy as np
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(random_state=random_state, solver='lbfgs', max_iter=5000)
logistic.fit(x_train, y_train)
logistic_prediction = logistic.predict(x_val)

print(f'[METRICS] Logistic Regression Classifier')
compute_metrics(y_val, logistic_prediction)

print(f'\n[CMATRIX] Logistic Regression Confusion Matrix')
conf_matrix(y_val, logistic_prediction, 1)